# Assignment 1 solution

This notebook contains a solution to assignment 1

## Imports

In [1]:
import random
import copy

import numpy as np

import nltk
import nltk.stem

import nltk.corpus
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

import sklearn
import sklearn.naive_bayes
import sklearn.ensemble
import sklearn.metrics
import sklearn.feature_extraction.text
import sklearn.utils
import sklearn.utils.testing
import sklearn.exceptions

random_state = 1111
np.random.seed(random_state)
random.seed(random_state)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Helgi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Helgi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Helgi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
E:\Repos\comp550\venv\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


## Pre-process data

In [2]:
def read_lines(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file_handler:
        return [sentence.rstrip() for sentence in file_handler.readlines()]

class Lemmatizer:
    def __init__(self):
        self.normalizer = nltk.stem.WordNetLemmatizer()
        self.tag_prefix_dict = {
            'J': nltk.corpus.wordnet.ADJ,
            'N': nltk.corpus.wordnet.NOUN,
            'V': nltk.corpus.wordnet.VERB,
            'R': nltk.corpus.wordnet.ADV
        }
    
    def __call__(self, document):
        tokens = nltk.word_tokenize(document)
        return [
            self.normalizer.lemmatize(token, pos=self.get_tag_class(tag))
            for token, tag in nltk.pos_tag(tokens)
        ]
    
    def get_tag_class(self, tag):
        prefix = tag[0].upper()
        return self.tag_prefix_dict.get(prefix, nltk.corpus.wordnet.NOUN)

class Stemmer:
    def __init__(self):
        self.normalizer = nltk.stem.PorterStemmer()
    
    def __call__(self, document):
        return [
            self.normalizer.stem(token)
            for token in nltk.word_tokenize(document)
        ]

def fit_vectorizer(X_data, tokenizer, stop_words, min_df):    
    vectorizer = sklearn.feature_extraction.text.CountVectorizer(
        tokenizer=tokenizer,
        stop_words=stop_words,
        min_df=min_df
    )
    vectorizer.fit_transform(X_data)
    return vectorizer

In [3]:
data_folder = 'E:/Repos/comp550/assignment1/data'
# data_folder = '/home/helgi/repos/comp550/assignment1/data/'
positives_file = 'rt-polarity.pos'
negatives_file = 'rt-polarity.neg'

splits = [0.8, 0.9]

negatives_data = [
    (document, 'negative')
    for document in read_lines(f'{data_folder}/{negatives_file}')
]
positives_data = [
    (document, 'positive')
    for document in read_lines(f'{data_folder}/{positives_file}')
]
all_data = sklearn.utils.shuffle(np.array(negatives_data + positives_data), random_state=random_state)
n = all_data.shape[0]

X_train_raw, y_train = all_data[:int(splits[0]*n), 0], all_data[:int(splits[0]*n), 1]
X_valid_raw, y_valid = all_data[int(splits[0]*n):int(splits[1]*n), 0], all_data[int(splits[0]*n): int(splits[1]*n), 1]
X_test_raw, y_test = all_data[int(splits[1]*n):, 0], all_data[int(splits[1]*n):, 1]

## Validation set hyper-parameter search across models

In [4]:
@sklearn.utils.testing.ignore_warnings(category=sklearn.exceptions.ConvergenceWarning)
def random_search(models, search_params, n_datasets=1, n_models=1):
    data_sets = []
    results = [{} for i in range(n_datasets)]

    data_set_variations = [
        choose_random_params(search_params['data'])
        for i in range(n_datasets)
    ]
    model_variations = {
        model_name: [choose_random_params(search_params['model'][model_name]) for i in range(n_models)]
        for model_name in search_params['model'].keys()
    }
    
    for i in range(n_datasets):
        print(f'Data set variation {i+1}/{n_datasets}')

        data_params = data_set_variations[i]
        data_sets.append(data_params)

        print('\tFitting vectorizer...')
        vectorizer = fit_vectorizer(X_train_raw, **data_params)

        X_train = vectorizer.transform(X_train_raw)
        X_valid = vectorizer.transform(X_valid_raw)
        X_test = vectorizer.transform(X_test_raw)

        for model_name, model_class in models.items():
            for j in range(n_models):
                print(f'\t{model_name} {j+1}/{n_models}')

                model_params = model_variations[model_name][j]

                model = model_class(**model_params)

                model.fit(X_train, y_train)
                valid_predictions = model.predict(X_valid)
                test_predictions = model.predict(X_test)
                
                valid_accuracy = sklearn.metrics.accuracy_score(y_valid, valid_predictions)
                
                # This number is only looked at once at the very end when the best models have been chosen based on validation accuracy
                test_accuracy = sklearn.metrics.accuracy_score(y_test, test_predictions)
                test_confusion_matrix = sklearn.metrics.confusion_matrix(y_test, test_predictions)

                if results[i].get(model_name, None) is None:
                    results[i][model_name] = []

                results[i][model_name].append({
                    'model_params': model_params,
                    'valid_accuracy': valid_accuracy,
                    'test_accuracy': test_accuracy,
                    'test_confusion_matrix': test_confusion_matrix
                })
    
    return data_sets, results

def choose_random_params(parameters):
    return {
        name: np.random.choice(values)
        for name, values in parameters.items()
    }

In [5]:
search_params = {
    'data': {
        'tokenizer': [Lemmatizer(), Stemmer()],
        'stop_words': ['english', None],
        'min_df': [1, 2, 3] # Minimum token frequency
    },
    'model': {
        'logistic_regression': {
            'eta0': [1e-3, 1e-2, 1e-1], # learning rate
            'alpha': [1e-3, 1e-2, 1e-1], # regularization
            'max_iter': np.arange(start=1, stop=5), # epochs
            'random_state': [random_state]
        },
        'linear_support_vector_machine': {
            'kernel': ['linear'],
            'max_iter': np.arange(start=1, stop=5), # epochs
            'C': [1e-3, 1e-2, 1e-1], # L2 regularization
            'random_state': [random_state]
        },
        'naive_bayes': {
            'alpha': np.arange(start=0.1, stop=1.1, step=0.1)
        },
        'random_forest': {
            'n_estimators': np.arange(start=10, stop=1000, step=10),
            'max_depth': np.append(np.array(None), np.arange(start=1, stop=5, step=2)),
            'random_state': [random_state]
        }
    }
}

models = {
    'logistic_regression': sklearn.linear_model.SGDClassifier,
    'linear_support_vector_machine': sklearn.svm.SVC,
    'naive_bayes': sklearn.naive_bayes.MultinomialNB,
    'random_forest': sklearn.ensemble.RandomForestClassifier
}

data_sets, results = random_search(models, search_params, n_datasets=3, n_models=4)

Data set variation 1/3
	Fitting vectorizer...
	logistic_regression 1/4
	logistic_regression 2/4
	logistic_regression 3/4
	logistic_regression 4/4
	linear_support_vector_machine 1/4
	linear_support_vector_machine 2/4
	linear_support_vector_machine 3/4
	linear_support_vector_machine 4/4
	naive_bayes 1/4
	naive_bayes 2/4
	naive_bayes 3/4
	naive_bayes 4/4
	random_forest 1/4
	random_forest 2/4
	random_forest 3/4
	random_forest 4/4
Data set variation 2/3
	Fitting vectorizer...


E:\Repos\comp550\venv\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wo

	logistic_regression 1/4
	logistic_regression 2/4
	logistic_regression 3/4
	logistic_regression 4/4
	linear_support_vector_machine 1/4
	linear_support_vector_machine 2/4
	linear_support_vector_machine 3/4
	linear_support_vector_machine 4/4
	naive_bayes 1/4
	naive_bayes 2/4
	naive_bayes 3/4
	naive_bayes 4/4
	random_forest 1/4
	random_forest 2/4
	random_forest 3/4
	random_forest 4/4
Data set variation 3/3
	Fitting vectorizer...


E:\Repos\comp550\venv\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


	logistic_regression 1/4
	logistic_regression 2/4
	logistic_regression 3/4
	logistic_regression 4/4
	linear_support_vector_machine 1/4
	linear_support_vector_machine 2/4
	linear_support_vector_machine 3/4
	linear_support_vector_machine 4/4
	naive_bayes 1/4
	naive_bayes 2/4
	naive_bayes 3/4
	naive_bayes 4/4
	random_forest 1/4
	random_forest 2/4
	random_forest 3/4
	random_forest 4/4


In [6]:
for i, params in enumerate(data_sets):
    print(f'\nDataset variation {i+1}: {params}')
    model_results = results[i]
    for model_name, params in model_results.items():
        print(f'\t{model_name}:')
        for j in range(len(params)):
            print('\t\t{}'.format({
                    'model_params': params[j]['model_params'],
                    'valid_accuracy': params[j]['valid_accuracy']
            }))



Dataset variation 1: {'tokenizer': <__main__.Lemmatizer object at 0x000001D60E7ABDC8>, 'stop_words': None, 'min_df': 2}
	logistic_regression:
		{'model_params': {'eta0': 0.001, 'alpha': 0.1, 'max_iter': 1, 'random_state': 1111}, 'valid_accuracy': 0.6050656660412758}
		{'model_params': {'eta0': 0.1, 'alpha': 0.001, 'max_iter': 3, 'random_state': 1111}, 'valid_accuracy': 0.7213883677298312}
		{'model_params': {'eta0': 0.1, 'alpha': 0.1, 'max_iter': 1, 'random_state': 1111}, 'valid_accuracy': 0.6050656660412758}
		{'model_params': {'eta0': 0.01, 'alpha': 0.1, 'max_iter': 2, 'random_state': 1111}, 'valid_accuracy': 0.6097560975609756}
	linear_support_vector_machine:
		{'model_params': {'kernel': 'linear', 'max_iter': 4, 'C': 0.1, 'random_state': 1111}, 'valid_accuracy': 0.48592870544090055}
		{'model_params': {'kernel': 'linear', 'max_iter': 1, 'C': 0.1, 'random_state': 1111}, 'valid_accuracy': 0.4821763602251407}
		{'model_params': {'kernel': 'linear', 'max_iter': 3, 'C': 0.001, 'random_

## Test set results

In [7]:
best_models = {
    'logistic_regression': 1,
    'naive_bayes': 0,
    'random_forest': 0
}

for model_name, model_index in best_models.items():
    print(f'Best {model_name} test sets accuracies:')
    for i in range(len(results)):
        print(f'\t{i+1}: {results[i][model_name][model_index]["test_accuracy"]}')

print(f'No best linear_support_vector_machine test sets accuracies')

random_predictions = np.random.choice(['negative', 'positive'], size=y_test.shape[0])
random_accuracy = sklearn.metrics.accuracy_score(y_test, random_predictions)
print(f'Random classifier test set accuracy: {random_accuracy}')

Best logistic_regression test sets accuracies:
	1: 0.7300843486410497
	2: 0.7319587628865979
	3: 0.7544517338331771
Best naive_bayes test sets accuracies:
	1: 0.7835051546391752
	2: 0.7497656982193065
	3: 0.7628865979381443
Best random_forest test sets accuracies:
	1: 0.6935332708528584
	2: 0.7216494845360825
	3: 0.7244611059044048
No best linear_support_vector_machine test sets accuracies
Random classifier test set accuracy: 0.5201499531396439


In [8]:
print('Best model (Naive Bayes) confusion matrix:')
print(results[0]['naive_bayes'][0]['test_confusion_matrix'])

Best model (Naive Bayes) confusion matrix:
[[433 110]
 [121 403]]
